In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

c:\Users\86177\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\86177\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\86177\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
bank_data = pd.read_csv("Proj1/data/bank-additional-full.csv",sep=";")
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
import matplotlib.pyplot as plt

qt_cols = bank_data.select_dtypes(include=['int64', 'float64']).columns
ql_cols = bank_data.iloc[:,:-1].select_dtypes(include=['object']).columns

In [4]:
new_table = bank_data

In [5]:
new_table = new_table[(new_table["education"]!="unknown") &\
                      (new_table["housing"]!="unknown")&\
                      (new_table["job"]!="unknown")&(new_table["marital"]!="unknown")]

In [6]:
new_table = new_table[new_table["duration"]!=0]

In [7]:
X = new_table[qt_cols]
for col in ql_cols:
    X = pd.concat([X, pd.get_dummies(new_table[col], prefix=col).iloc[:,:-1]], axis = 1)

y = new_table.iloc[:,-1].map({'yes':1, 'no':0})

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

feature_list = [X.columns]

In [48]:
X_train.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'marital_divorced', 'marital_married',
       'education_basic.4y', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'default_no', 'default_unknown',
       'housing_no', 'loan_no', 'contact_cellular', 'month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'day_of_week_fri', 'day_of_week_mon',
       'day_of_week_thu', 'day_of_week_tue', 'poutcome_failure',
       'poutcome_nonexistent'],
      dtype='object')

In [60]:
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import xgboost
import time

# Training the XGboost model
xgb_classifier = xgboost.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
# Start the timer
start_time = time.time()
xgb_classifier.fit(X_train, y_train)
print(time.time()-start_time)
# Predictions and evaluation
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

accuracy 


1.6007812023162842


0.913797611784189

In [50]:
# Calculating recall score
recall = recall_score(y_test, y_pred)

recall

0.5320197044334976

In [51]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)

0.6506024096385542

In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      6837
           1       0.65      0.53      0.59       812

    accuracy                           0.92      7649
   macro avg       0.80      0.75      0.77      7649
weighted avg       0.91      0.92      0.92      7649



In [12]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# Explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Calculate cumulative variance
cumulative_variance = np.cumsum(explained_variance)

# Determine the number of components to reach at least 95% of explained variance
n_components = np.where(cumulative_variance >= 0.95)[0][0] + 1

In [54]:
pd.Series(cumulative_variance)

0     0.633636
1     0.957881
2     0.998697
3     0.999692
4     0.999883
5     0.999953
6     0.999961
7     0.999965
8     0.999967
9     0.999970
10    0.999972
11    0.999975
12    0.999977
13    0.999979
14    0.999981
15    0.999983
16    0.999985
17    0.999986
18    0.999988
19    0.999989
20    0.999991
21    0.999992
22    0.999993
23    0.999994
24    0.999994
25    0.999995
26    0.999996
27    0.999996
28    0.999997
29    0.999997
30    0.999998
31    0.999998
32    0.999998
33    0.999999
34    0.999999
35    0.999999
36    0.999999
37    1.000000
38    1.000000
39    1.000000
40    1.000000
41    1.000000
42    1.000000
43    1.000000
44    1.000000
45    1.000000
46    1.000000
47    1.000000
dtype: float64

In [55]:
# Reduce the dataset dimensions with PCA to the selected number of components
pca_reduced = PCA(n_components=n_components)
X_pca_reduced = pca_reduced.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca_reduced, y, test_size=0.3, random_state=42)

xgb_classifier = xgboost.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
xgb_classifier.fit(X_train, y_train)
# Predictions and evaluation
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


classification_rep = classification_report(y_test, y_pred)

n_components, accuracy

(2, 0.913797611784189)

In [56]:
print(classification_rep)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95     10216
           1       0.65      0.47      0.54      1257

    accuracy                           0.91     11473
   macro avg       0.79      0.72      0.75     11473
weighted avg       0.91      0.91      0.91     11473



In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

xgb_classifier = xgboost.XGBClassifier(use_label_encoder=False, 
                                       eval_metric='logloss')

# Create a GridSearchCV object
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9]
}

grid_search = GridSearchCV(estimator=xgb_classifier, 
                           param_grid=param_grid, 
                           cv=3, 
                           scoring='accuracy', 
                           verbose=2)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Use the best estimator and the best score
accuracy = grid_search.score(X_test, y_test)
print("Test set accuracy: {:.2f}".format(accuracy))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .......................max_depth=3, n_estimators=50; total time=   0.3s
[CV] END .......................max_depth=3, n_estimators=50; total time=   0.2s
[CV] END .......................max_depth=3, n_estimators=50; total time=   0.3s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.4s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.5s
[CV] END ......................max_depth=3, n_estimators=100; total time=   0.5s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.0s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.0s
[CV] END ......................max_depth=3, n_estimators=200; total time=   1.0s
[CV] END .......................max_depth=6, n_estimators=50; total time=   0.4s
[CV] END .......................max_depth=6, n_estimators=50; total time=   0.4s
[CV] END .......................max_depth=6, n_es

In [59]:
import time
xgb_classifier = xgboost.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss', n_estimators = 50, max_depth = 3)

# Start the timer
start_time = time.time()

# Fit the model
xgb_classifier.fit(X_train, y_train)

# Measure the duration
duration = time.time() - start_time
print(f"Training time: {duration:.2f} seconds")

# Predictions and evaluation
y_pred = xgb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


classification_rep = classification_report(y_test, y_pred)
print(classification_rep)

Training time: 0.45 seconds
              precision    recall  f1-score   support

           0       0.94      0.97      0.95     10216
           1       0.66      0.48      0.55      1257

    accuracy                           0.92     11473
   macro avg       0.80      0.72      0.75     11473
weighted avg       0.91      0.92      0.91     11473

